---
    
### **Introduction to the Task Management Assistant Capstone Project**

Welcome to the Task Management Assistant Capstone Project! In this project, you'll learn how to effectively utilize the Swarmauri SDK to integrate large language model (LLM) tools and function calling into a task management application. This capstone challenges you to develop a system that leverages Swarmauri's capabilities to build intelligent, interactive tools that can process natural language inputs, manage tasks, and provide meaningful responses to users.

#### **What You'll Learn**
This project is designed to provide hands-on experience with:
- **Swarmauri SDK Integration**: Learning how to set up and use the Swarmauri SDK to build interactive task management tools.

- **LLM Tools and Function Calling**: Implementing function calls with large language models to enhance task management functionalities with the help of swarmauri sdk components.

#### **Who Is This For?**
This project is ideal for:
- Developers with a foundational knowledge of Python.
- Individuals seeking practical experience in leveraging large language models within their projects.
- Aspiring engineers interested in building intelligent tools that utilize advanced AI functionalities for productivity applications.

#### **Outcome**
Upon completion, you'll have a fully functional Task Management Assistant that demonstrates how to integrate the Swarmauri SDK, its LLM tools/function calling into a real-world application. You'll gain valuable insights into using AI-driven tools to enhance task management and user interactions.


---
### **1. Importing Required Libraries**
Import essential libraries for date manipulation, type annotations, Google APIs, and SwarmAURI SDK components.

In [1]:
from datetime import datetime, timedelta
from dateutil import parser
from typing import Optional, Dict, Any, List
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import os.path
import pickle
from swarmauri.tools.base.ToolBase import ToolBase
from swarmauri.tools.base.ParameterBase import ParameterBase as Parameter

---

### **2. Define `GoogleWorkspaceManager` Class**
A manager class to handle Google Calendar and Google Tasks operations
with unified authentication and service initialization.

It has the following methods:

- #### **Initialize Services**
    - Set up Google Calendar and Tasks services, handling authentication 
using OAuth 2.0 and saving credentials for reuse.

- #### **Get or Create Default Task List**
    - Retrieve the default Google Tasks list or create one if none exists.

- #### **Create a Task in Google Tasks**
    - Create a task with optional due date and notes, ensuring proper formatting.

- #### **List Tasks**
    - Retrieve tasks from the default task list, with options to filter by number.

- #### **Update a Task's Status**
    - Mark a task as completed or update its status.

- #### **Create a Calendar Event**
    - Add an event to Google Calendar with title, start time, duration, and description.


In [2]:

class GoogleWorkspaceManager:
    """
    Manages both Google Calendar and Google Tasks operations.
    This unified manager ensures consistent authentication and resource handling.
    """
    SCOPES = [
        'https://www.googleapis.com/auth/calendar',
        'https://www.googleapis.com/auth/tasks'
    ]
    
    def __init__(self):
        self.creds = None
        self.calendar_service = None
        self.tasks_service = None
        self.initialize_services()

    def initialize_services(self):
        """Initialize both Calendar and Tasks services with proper authentication"""
        # Handle authentication for both services
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                self.creds = pickle.load(token)
        
        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json',
                    self.SCOPES
                )
                self.creds = flow.run_local_server(port=8000)
                with open('token.pickle', 'wb') as token:
                    pickle.dump(self.creds, token)
        
        # Initialize both services
        self.calendar_service = build('calendar', 'v3', credentials=self.creds)
        self.tasks_service = build('tasks', 'v1', credentials=self.creds)
        
        # Get or create default task list
        self.default_tasklist = self._get_or_create_default_tasklist()

    def _get_or_create_default_tasklist(self) -> str:
        """Get the default task list ID or create one if it doesn't exist"""
        try:
            tasklists = self.tasks_service.tasklists().list().execute()
            if tasklists.get('items'):
                return tasklists['items'][0]['id']
            else:
                # Create a new task list
                new_tasklist = self.tasks_service.tasklists().insert(
                    body={'title': 'My Tasks'}
                ).execute()
                return new_tasklist['id']
        except Exception as e:
            raise Exception(f"Failed to initialize task list: {str(e)}")

    
    def create_task(self, title: str, due_date: Optional[str] = None, 
                    notes: Optional[str] = None) -> Dict:
        """Create a new task in Google Tasks"""
        try:
            task_body = {
                'title': title,
                'notes': notes or ''
            }
            
            if due_date:
                # Convert due date to RFC 3339 format
                due_datetime = parser.parse(due_date)
                # Google Tasks requires due date to be set to midnight UTC
                due_datetime = due_datetime.replace(hour=0, minute=0, second=0, microsecond=0)
                task_body['due'] = due_datetime.isoformat() + 'Z'
            
            result = self.tasks_service.tasks().insert(
                tasklist=self.default_tasklist,
                body=task_body
            ).execute()
            
            return {
                'status': 'success',
                'task_id': result['id'],
                'task': result
            }
        except Exception as e:
            return {
                'status': 'error',
                'message': f"Failed to create task: {str(e)}"
            }
        
    def list_tasks(self, max_results: int = 100) -> Dict:
        """List all tasks from the default task list"""
        try:
            result = self.tasks_service.tasks().list(
                tasklist=self.default_tasklist,
                maxResults=max_results,
                showCompleted=True
            ).execute()
            
            return {
                'status': 'success',
                'tasks': result.get('items', [])
            }
        except Exception as e:
            return {
                'status': 'error',
                'message': f"Failed to list tasks: {str(e)}"
            }
        
    def update_task(self, task_id: str, status: str = 'completed') -> Dict:
        """Update a task's status"""
        try:
            # First get the current task
            task = self.tasks_service.tasks().get(
                tasklist=self.default_tasklist,
                task=task_id
            ).execute()
            
            # Update the status
            task['status'] = status
            
            result = self.tasks_service.tasks().update(
                tasklist=self.default_tasklist,
                task=task_id,
                body=task
            ).execute()
            
            return {
                'status': 'success',
                'task': result
            }
        except Exception as e:
            return {
                'status': 'error',
                'message': f"Failed to update task: {str(e)}"
            }
        
    def create_event(self, title: str, start_time: str, 
                     duration: int, description: Optional[str] = None) -> Dict:
        """Create a calendar event (keeping our previous calendar functionality)"""
        start_datetime = datetime.strptime(start_time, "%Y-%m-%d %H:%M")
        end_datetime = start_datetime + timedelta(minutes=duration)
        
        event = {
            'summary': title,
            'description': description,
            'start': {
                'dateTime': start_datetime.isoformat(),
                'timeZone': 'UTC',
            },
            'end': {
                'dateTime': end_datetime.isoformat(),
                'timeZone': 'UTC',
            }
        }
        
        try:
            event_result = self.calendar_service.events().insert(
                calendarId='primary',
                body=event
            ).execute()
            return {
                'status': 'success',
                'event_id': event_result['id'],
                'html_link': event_result['htmlLink'],
                'event': event_result
            }
        except Exception as e:
            return {
                'status': 'error',
                'message': f"Failed to create event: {str(e)}"
            }


#### **Initialize our workspace manager**

In [3]:
workspace_manager = GoogleWorkspaceManager()

---
### **3. Define the Tools for Task Management**
#### **Create Event Tool**
A tool to create events in Google Calendar using parameters like title, start time, duration, and optional description.

In [4]:

from typing import Literal


class CreateEventTool(ToolBase):
    name: str = "CreateEventTool"
    description: str = "Creates a new calendar event in Google Calendar"
    type: Literal['CreateEventTool'] = 'CreateEventTool'
    parameters: List[Parameter] = [
        Parameter(
            name="title",
            description="The title of the event",
            type="string",
            required=True
        ),
        Parameter(
            name="start_time",
            description="Start time in YYYY-MM-DD HH:MM format",
            type="string",
            required=True
        ),
        Parameter(
            name="duration",
            description="Duration in minutes",
            type="integer",
            required=True
        ),
        Parameter(
            name="description",
            description="Optional event description",
            type="string",
            required=False
        )
    ]

    def __call__(self, title: str, start_time: str, duration: int, 
                 description: Optional[str] = None) -> Dict[str, Any]:
        try:
            result = workspace_manager.create_event(title, start_time, duration, description)
            if result['status'] == 'success':
                return {
                    "status": "success",
                    "event": result['event'],
                    "message": f"Calendar event '{title}' scheduled successfully",
                    "calendar_link": result['html_link']
                }
            return result
        except ValueError as e:
            return {
                "status": "error",
                "message": f"Invalid date format. Please use YYYY-MM-DD HH:MM format. Error: {str(e)}"
            }


#### **Complete Task Tool**
A tool to mark tasks as completed using their unique task ID.

In [5]:

class CreateTaskTool(ToolBase):
    name: str = "CreateTaskTool"
    description: str = "Creates a new task in Google Tasks"
    type: Literal['CreateTaskTool'] = 'CreateTaskTool'
    parameters: List[Parameter] = [
        Parameter(
            name="title",
            description="The title of the task",
            type="string",
            required=True
        ),
        Parameter(
            name="due_date",
            description="Optional due date in YYYY-MM-DD HH:MM format",
            type="string",
            required=False

        ),
        Parameter(
            name="notes",
            description="Optional notes for the task",
            type="string",
            required=False
        )
    ]

    def __call__(self, title: str, due_date: Optional[str] = None, 
                 notes: Optional[str] = None) -> Dict[str, Any]:
        try:
            result = workspace_manager.create_task(title, due_date, notes)
            if result['status'] == 'success':
                return {
                    "status": "success",
                    "task": result['task'],
                    "message": f"Task '{title}' created successfully"
                }
            print(result)
            return result
        except ValueError as e:
            print(f"Error: {str(e)}")
            return {
                "status": "error",
                "message": f"Invalid date format. Please use YYYY-MM-DD HH:MM format. Error: {str(e)}"
            }


#### **List Tasks Tool**
A tool to retrieve tasks from Google Tasks with configurable maximum results.

In [6]:

class ListTasksTool(ToolBase):
    name: str = "ListTasksTool"
    description: str = "Lists all tasks from Google Tasks"
    type: Literal['ListTasksTool'] = 'ListTasksTool'
    parameters: List[Parameter] = [
        Parameter(
            name="max_results",
            description="Maximum number of tasks to return",
            type="integer",
            required=False
        )
    ]

    def __call__(self, max_results: int = 100) -> Dict[str, Any]:
        result = workspace_manager.list_tasks(max_results)
        if result['status'] == 'success':
            tasks = result['tasks']
            return {
                "status": "success",
                "tasks": tasks,
                "message": f"Found {len(tasks)} tasks"
            }
        return result


#### **Complete Task Tool**
A tool to mark tasks as completed using their unique task ID.

In [7]:

class CompleteTaskTool(ToolBase):
    name: str = "CompleteTaskTool"
    description: str = "Marks a Google Task as completed"
    type: Literal['CompleteTaskTool'] = 'CompleteTaskTool'
    parameters: List[Parameter] = [
        Parameter(
            name="task_name",
            description="The ID of the task to complete",
            type="string",
            required=True
        )
    ]

    def __call__(self, task_id: str) -> Dict[str, Any]:
        result = workspace_manager.update_task(task_id, 'completed')
        if result['status'] == 'success':
            return {
                "status": "success",
                "task": result['task'],
                "message": "Task marked as completed"
            }
        return result

---
### **4. Setting Up Swarmauri SDK Toolkit**
Integrate tools with the Swarmauri SDK toolkit for streamlined execution.

In [10]:
from swarmauri.llms.concrete.OpenAIToolModel import OpenAIToolModel
from swarmauri.agents.concrete.ToolAgent import ToolAgent
from swarmauri.conversations.concrete.Conversation import Conversation
from swarmauri.toolkits.concrete.Toolkit import Toolkit

# Import the OpenAI API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

toolkit = Toolkit(
    tools= {
        "CreateTaskTool": CreateTaskTool(),
        "CreateEventTool": CreateEventTool(),
        "CompleteTaskTool": CompleteTaskTool(),
        "ListTasksTool": ListTasksTool()
    }
)


# Initialize a mock LLM
llm = OpenAIToolModel(api_key=OPENAI_API_KEY)

# Initialize a conversation manager
conversation = Conversation()

agent = ToolAgent(
    llm=llm,
    toolkit=toolkit,
    conversation=conversation
)


### **Create the Gradio interface**

In [9]:
import gradio as gr

def converse(input_text, history):
    # Ensure input_text is a string
    input_text = str(input_text)
    # Execute the input command with the ToolAgent
    result = agent.exec(input_text)

    return result

# Set up the Gradio ChatInterface
demo = gr.ChatInterface(
    fn=converse,
    examples=["Schedule a meeting with John by 11-12-2021 10:00 AM", "Create a task to buy groceries", "List all tasks"],
    title="🐝 Task Management Assistant",
    description="A Task Management Assistant! I can help you schedule events and manage tasks. Try me out!",
)

if __name__ == "__main__":
    demo.launch()

/Users/michaeldecent/miniconda3/envs/swarmauri_notebook_env/lib/python3.11/site-packages/gradio/components/chatbot.py:248: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
